<a href="https://colab.research.google.com/github/jiinpark21/ESAA_24-2/blob/main/9_13_%EC%84%B8%EC%85%98_%EB%AA%A8%EB%8D%B8%ED%9B%88%EB%A0%A8_%EC%97%B0%EC%8A%B5%EB%AC%B8%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **모델 훈련 연습 문제**
___
- 출처 : 핸즈온 머신러닝 Ch04 연습문제 1, 5, 9, 10
- 개념 문제의 경우 텍스트 셀을 추가하여 정답을 적어주세요.

### **1. 수백만 개의 특성을 가진 훈련 세트에서는 어떤 선형 회귀 알고리즘을 사용할 수 있을까요?**
___


특성이 많은 훈련 세트에서는 경사 하강법을 사용할 수 있으며, 경사 하강법의 종류로는 확률적 경사 하강법, 미니배치 경사 하강법 등이 있다.

### **2. 배치 경사 하강법을 사용하고 에포크마다 검증 오차를 그래프로 나타내봤습니다. 검증 오차가 일정하게 상승되고 있다면 어떤 일이 일어나고 있는 걸까요? 이 문제를 어떻게 해결할 수 있나요?**
___

에포크마다 검증 오차가 일정하게 상승하고 있다면 학습률이 너무 높아 알고리즘이 발산하는 것일 수 있다. 이때 훈련 에러도 같이 올라간다면 학습률을 낮추어야 하고, 훈련 에러는 증가하지 않는다면 모델이 훈련 세트에 과대적합된 것이므로 훈련을 멈추어야 한다.

### **3. 릿지 회귀를 사용했을 때 훈련 오차가 검증 오차가 거의 비슷하고 둘 다 높았습니다. 이 모델에는 높은 편향이 문제인가요, 아니면 높은 분산이 문제인가요? 규제 하이퍼파라미터 $\alpha$를 증가시켜야 할까요 아니면 줄여야 할까요?**
___

훈련 오차와 검증 오차가 거의 비슷하고 높다면, 모델이 훈련 세트에 과소적합되었을 가능성이 높다. 이는 높은 편향을 가진 모델을 뜻하므로, 규제 하이퍼파라미터 α를 줄여야 한다.

### **4. 다음과 같이 사용해야 하는 이유는?**
___
- 평범한 선형 회귀(즉, 아무런 규제가 없는 모델) 대신 릿지 회귀
- 릿지 회귀 대신 라쏘 회귀
- 라쏘 회귀 대신 엘라스틱넷

- 일반적으로 규제가 있는 모델이 규제가 없는 모델보다 성능이 좋다.
- 라쏘 회귀는 $l_{1}$ 페널티를 통해 가중치를 0으로 만들어, 중요한 특성만 남기는 특성 선택의 효과를 가진다. 일부 특성만 유용할 것 같을 때 라쏘를 사용하면 좋으며, 확신이 없을 때는 릿지 회귀를 사용한다.
- 라쏘가 불규칙하게 행동하는 경우가 있으므로, 일반적으로 엘라스틱넷이 라쏘보다 선호된다.

### **추가) 조기 종료를 사용한 배치 경사 하강법으로 iris 데이터를 활용해 소프트맥스 회귀를 구현해보세요(사이킷런은 사용하지 마세요)**


---



In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from sklearn.datasets import load_iris

iris = load_iris()

In [3]:
X = iris['data'][:, (2,3)]
y = iris['target']

In [4]:
# 샘플에 편향 추가 (X0=1)
X_bias = np.c_[np.ones([len(X), 1]), X]

In [6]:
np.random.seed(100)

In [7]:
# 데이터셋 나누기
test_ratio = 0.2
validation_ratio = 0.2
total_size = len(X_bias)

test_size = int(total_size * test_ratio)
validation_size = int(total_size * validation_ratio)
train_size = total_size - test_size - validation_size

rnd_indices = np.random.permutation(total_size)

X_train = X_bias[rnd_indices[:train_size]]
y_train = y[rnd_indices[:train_size]]
X_valid = X_bias[rnd_indices[train_size:-test_size]]
y_valid = y[rnd_indices[train_size:-test_size]]
X_test = X_bias[rnd_indices[-test_size:]]
y_test = y[rnd_indices[-test_size:]]

In [8]:
# 원-핫 벡터로 바꾸는 함수
def to_one_hot(y):
    n_classes = y.max() + 1
    m = len(y)
    Y_one_hot = np.zeros((m, n_classes))
    Y_one_hot[np.arange(m), y] = 1
    return Y_one_hot

In [9]:
# 훈련 세트, 테스트 세트 -> 타깃 클래스 확률 담은 행렬
Y_train_one_hot = to_one_hot(y_train)
Y_valid_one_hot = to_one_hot(y_valid)
Y_test_one_hot = to_one_hot(y_test)

In [10]:
# 소프트맥스 함수 구현
def softmax(logits):
    exps = np.exp(logits)
    exp_sums = np.sum(exps, axis=1, keepdims=True)
    return exps / exp_sums

In [11]:
# 입출력 개수 정의
n_inputs = X_train.shape[1] # == 3 (특성 2개와 편향)
n_outputs = len(np.unique(y_train))   # == 3 (3개의 붓꽃 클래스)

In [12]:
# 비용함수 구현
eta = 0.01
n_iterations = 5001
m = len(X_train)
epsilon = 1e-7

Theta = np.random.randn(n_inputs, n_outputs)

for iteration in range(n_iterations):
    logits = X_train.dot(Theta)
    Y_proba = softmax(logits)
    if iteration % 500 == 0:
        loss = -np.mean(np.sum(Y_train_one_hot * np.log(Y_proba + epsilon), axis=1))
        print(iteration, loss)
    error = Y_proba - Y_train_one_hot
    gradients = 1/m * X_train.T.dot(error)
    Theta = Theta - eta * gradients

0 2.772690913838786
500 0.7318861605764747
1000 0.6145718155013726
1500 0.5445794516872344
2000 0.49825747589294295
2500 0.4649747647571402
3000 0.43956173778465363
3500 0.4192686369999683
4000 0.4025116424095841
4500 0.3883156939664539
5000 0.3760465936872414


In [13]:
Theta

array([[ 3.33239344, -1.00805654, -2.77173136],
       [-1.17180011,  0.46444169, -0.0404394 ],
       [ 0.63037944,  0.4201552 ,  3.16409623]])

In [14]:
# 검증 세트 예측과 정확도
logits = X_valid.dot(Theta)
Y_proba = softmax(logits)
y_predict = np.argmax(Y_proba, axis=1)

accuracy_score = np.mean(y_predict == y_valid)
accuracy_score

0.9

In [15]:
# 조기 종료 추가
eta = 0.1
n_iterations = 5001
m = len(X_train)
epsilon = 1e-7
alpha = 0.1  # 규제 하이퍼파라미터
best_loss = np.infty

Theta = np.random.randn(n_inputs, n_outputs)

for iteration in range(n_iterations):
    logits = X_train.dot(Theta)
    Y_proba = softmax(logits)
    error = Y_proba - Y_train_one_hot
    gradients = 1/m * X_train.T.dot(error) + np.r_[np.zeros([1, n_outputs]), alpha * Theta[1:]]
    Theta = Theta - eta * gradients

    logits = X_valid.dot(Theta)
    Y_proba = softmax(logits)
    xentropy_loss = -np.mean(np.sum(Y_valid_one_hot * np.log(Y_proba + epsilon), axis=1))
    l2_loss = 1/2 * np.sum(np.square(Theta[1:]))
    loss = xentropy_loss + alpha * l2_loss
    if iteration % 500 == 0:
        print(iteration, loss)
    if loss < best_loss:
        best_loss = loss
    else:
        print(iteration - 1, best_loss)
        print(iteration, loss, "조기 종료!")
        break

0 3.803859836647828
500 0.5376199083872859
1000 0.508904063009899
1500 0.5017939363942643
2000 0.499576114412414
2500 0.49892179941182924
2998 0.4988024379346561
2999 0.4988024384928676 조기 종료!


In [16]:
# 조기 종료 추가한 코드에서의 예측과 정확도
logits = X_valid.dot(Theta)
Y_proba = softmax(logits)
y_predict = np.argmax(Y_proba, axis=1)

accuracy_score = np.mean(y_predict == y_valid)
accuracy_score

0.9333333333333333